In [1]:
import sys
import os

sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this


In [2]:
from automl.meta_rl.hyperparameter_suggestion import HyperparameterSuggestion


hyperparameters_to_change : list[HyperparameterSuggestion] = []

c:\Users\ricar\.conda\envs\aa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
hyperparameters_to_change = [ *hyperparameters_to_change, 
                                
                             HyperparameterSuggestion(
                                name='num_episodes', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["num_episodes"])
                                ],
                                #value_suggestion = ('int', {'low':200, 'high':800, 'step':100}) 
                                value_suggestion = ('int', {'low':2000, 'high':2000}) 
                            ),
                            ]       

In [4]:
hyperparameters_to_change = [ *hyperparameters_to_change, 
                                
                            HyperparameterSuggestion(
                                name='discount_factor', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["agents_input", "discount_factor"])
                                ],
                                value_suggestion = ('float', {'low':0.5, 'high':0.99}) 
                            )
                            ]       

In [5]:

hyperparameters_to_change = [ *hyperparameters_to_change, 

                             HyperparameterSuggestion(
                                name='epsilon_start', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["agents_input", "exploration_strategy_input", "epsilon_start"])
                                ],
                                value_suggestion = ('float', {'low':0.95, 'high':0.999}) 
                            ),
                             HyperparameterSuggestion(
                                name='epsilon_end', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["agents_input", "exploration_strategy_input", "epsilon_end"])
                                ],
                                value_suggestion = ('float', {'low':0.05, 'high':0.3}) 
                            ),
                             HyperparameterSuggestion(
                                name='epsilon_decay', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["agents_input", "exploration_strategy_input", "epsilon_decay"])
                                ],
                                value_suggestion = ('float', {'low':0.95, 'high':0.9999}) 
                             )
                             
]


In [6]:
hyperparameters_to_change = [ *hyperparameters_to_change, 
                             
                             HyperparameterSuggestion(
                                name='hidden_layers', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["agents_input", "model_input", "hidden_layers"])
                                ],
                                value_suggestion = ('int', {'low':2, 'high':5}) 
                            ),
                             HyperparameterSuggestion(
                                name='hidden_size', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["agents_input", "model_input", "hidden_size"])
                                ],
                                value_suggestion = ('cat', {'choices' : [16, 32, 64, 128]}) 
                            )
]

In [7]:
hyperparameters_to_change = [ *hyperparameters_to_change, 
                             
                             HyperparameterSuggestion(
                                name='target_update_rate', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["agents_input", "learner_input", "target_update_rate"])
                                ],
                                value_suggestion = ('float', {'low':0.01, 'high':0.15}) 
                            ),
                             HyperparameterSuggestion(
                                name='learning_rate', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["agents_input", "learner_input", "optimizer_input", "learning_rate"])
                                ],
                                value_suggestion = ('float', {'low':0.000001, 'high':0.1}) 
                            )            
]

In [8]:
hyperparameters_to_change = [ *hyperparameters_to_change, 
                             
                             HyperparameterSuggestion(
                                name='memory_capacity', 
                                hyperparameter_localizations= [
                                    ('RLPipelineComponent', ["agents_input", "memory_input", "capacity"])
                                ],
                                value_suggestion = ('int', {'low':100, 'high':1000}) 
                            )
                           
]

In [9]:
from automl.base_configurations.base_configurations import load_configuration_dict

hp_optimization_input = {
    "configuration_dict" : load_configuration_dict('basic_rl'),
    "hyperparameters_range_list" : hyperparameters_to_change,
    "n_trials" : 20
    }

In [10]:
from automl.meta_rl.hp_optimization_pipeline import HyperparameterOptimizationPipeline

hp_optimization_pipeline = HyperparameterOptimizationPipeline(hp_optimization_input)

In [11]:
hp_optimization_pipeline.run()

[I 2025-03-01 15:19:56,241] A new study created in memory with name: no-name-3f286d31-cdae-4f10-9beb-ec6a71dd3cda


generate_log_directory for object HyperparameterOptimizationPipeline
Opening a log in directory: data\experiments\HyperparameterOptimizationPipeline_17, with name:
Type of object with name: <class 'automl.meta_rl.hp_optimization_pipeline.HyperparameterOptimizationPipeline'> and name passed: 
HyperparameterOptimizationPipeline: Created logger in directory: data\experiments\HyperparameterOptimizationPipeline_17
HyperparameterOptimizationPipeline: Hyperparameter names: ['num_episodes', 'discount_factor', 'epsilon_start', 'epsilon_end', 'epsilon_decay', 'hidden_layers', 'hidden_size', 'target_update_rate', 'learning_rate', 'memory_capacity']
HyperparameterOptimizationPipeline: Creating component to test
Opening a log in directory: data\experiments\HyperparameterOptimizationPipeline_17, with name:configuration_1
Log directory did not exist, creating it at: data\experiments\HyperparameterOptimizationPipeline_17\configuration_1
HyperparameterOptimizationPipeline: Created component with name c

c:\Users\ricar\Universidade\Tese\Tese\rep\ricardo-goncalo-thesis-project\project\automl\loggers\result_logger.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.dataframe = pandas.concat((self.dataframe, results_df), ignore_index=True)
c:\Users\ricar\Universidade\Tese\Tese\rep\ricardo-goncalo-thesis-project\project\automl\loggers\result_logger.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.dataframe = pandas.concat((self.dataframe, results_df), ignore_index=True)
c:\Users\ricar

AgentTrainer: Ended episode: 2 with duration: 30, total reward: 3.222222222222223
AgentTrainer: Ended episode: 2 with duration: 30, total reward: -6.777777777777777
AgentTrainer: Values of exploraion strategy: {'n_random': 46, 'n_greedy': 18}
AgentTrainer: Values of exploraion strategy: {'n_random': 49, 'n_greedy': 15}
Last results: {'episode': 2, 'episode_steps': 60, 'avg_reward': -0.059259259259259386, 'total_reward': -3.5555555555555634}
Result: -3.5555555555555634
Type of object with name: <class 'automl.loggers.result_logger.ResultLogger'> and name passed: 
ResultLogger: Created logger in directory: data\experiments\HyperparameterOptimizationPipeline_17
HyperparameterOptimizationPipeline: Creating component to test
Opening a log in directory: data\experiments\HyperparameterOptimizationPipeline_17, with name:configuration_2
Log directory did not exist, creating it at: data\experiments\HyperparameterOptimizationPipeline_17\configuration_2
HyperparameterOptimizationPipeline: Created 

c:\Users\ricar\Universidade\Tese\Tese\rep\ricardo-goncalo-thesis-project\project\automl\loggers\result_logger.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.dataframe = pandas.concat((self.dataframe, results_df), ignore_index=True)
c:\Users\ricar\Universidade\Tese\Tese\rep\ricardo-goncalo-thesis-project\project\automl\loggers\result_logger.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.dataframe = pandas.concat((self.dataframe, results_df), ignore_index=True)
c:\Users\ricar

...

[I 2025-03-01 15:20:52,862] Trial 5 finished with value: -111.00000000000215 and parameters: {'num_episodes': 2, 'discount_factor': 0.9506285495815807, 'epsilon_start': 0.9533128955452946, 'epsilon_end': 0.13722470346293814, 'epsilon_decay': 0.9612905941787039, 'hidden_layers': 5, 'hidden_size': 16, 'target_update_rate': 0.14059334819892394, 'learning_rate': 0.06407080077096763, 'memory_capacity': 960}. Best is trial 5 with value: -111.00000000000215.


RLTrainerComponent: In episode 1, reached step 1000 that is beyond the current limit, 1000
AgentTrainer: Ended episode: 2 with duration: 500, total reward: 55.44444444444501
AgentTrainer: Ended episode: 2 with duration: 500, total reward: 55.555555555556126
AgentTrainer: Values of exploraion strategy: {'n_random': 283, 'n_greedy': 277}
AgentTrainer: Values of exploraion strategy: {'n_random': 274, 'n_greedy': 286}
Last results: {'episode': 2, 'episode_steps': 1000, 'avg_reward': 0.11100000000000215, 'total_reward': 111.00000000000215}
Result: 111.00000000000215
HyperparameterOptimizationPipeline: Creating component to test
Opening a log in directory: data\experiments\HyperparameterOptimizationPipeline_17, with name:configuration_7
Log directory did not exist, creating it at: data\experiments\HyperparameterOptimizationPipeline_17\configuration_7
HyperparameterOptimizationPipeline: Created component with name configuration_7
HyperparameterOptimizationPipeline: Starting new training with 

[W 2025-03-01 15:20:54,143] Trial 6 failed with parameters: {'num_episodes': 2, 'discount_factor': 0.9501324956742558, 'epsilon_start': 0.9949271191672096, 'epsilon_end': 0.2991417948751483, 'epsilon_decay': 0.9649965258781966, 'hidden_layers': 5, 'hidden_size': 32, 'target_update_rate': 0.08648524663727791, 'learning_rate': 0.09167816710133085, 'memory_capacity': 109} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\ricar\.conda\envs\aa\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "c:\Users\ricar\Universidade\Tese\Tese\rep\ricardo-goncalo-thesis-project\project\automl\meta_rl\hp_optimization_pipeline.py", line 176, in <lambda>
    study.optimize( lambda trial : self.objective(trial),
  File "c:\Users\ricar\Universidade\Tese\Tese\rep\ricardo-goncalo-thesis-project\project\automl\meta_rl\hp_optimization_pipeline.py", line 144, in objective
    component_to_test.train

Initializing model with input{'hidden_layers': 3, 'hidden_size': 64, 'input_shape': (2, torch.Size([3, 280, 480])), 'output_shape': Discrete(3), 'device': device(type='cuda')}


KeyboardInterrupt: 

In [ ]:
hp_optimization_pipeline.save_configuration()

In [ ]:
experiment_path = hp_optimization_pipeline.lg.logDir

print(experiment_path)

data\experiments\HyperparameterOptimizationPipeline_12
